In [4]:
import sys
import os
import pandas as pd
import pickle

sys.path.append("../code/")
from model import read_edgelists_from_df
from model_analysis import get_group_durations, get_group_times
from model_analysis import get_dis_agg_matrices, get_full_dis_agg_matrices, dis_agg_matrix_to_df
from utils import get_cumulative_Gs_from_Hs, get_groups_dict_from_Hs
from data_analysis import measure_social_memory, get_interevent_times

Reading the dataframe with the parameters and simulation IDs

In [2]:
#Reading the parameter dataframe
PATH = 'results/'

pars_df = pd.read_csv(PATH+'parameters.csv')
pars_df.set_index('pars_id', inplace=True)
pars_df.head()

,N,t_max,beta,alpha,n0,L,epsilon
pars_id,,,,,,,
0,700,2000,0.8,0.05,3.0,1.0,1
1,700,2000,0.8,0.05,3.0,1.0,5
2,700,2000,0.8,0.05,3.0,1.0,10
3,700,2000,0.8,0.05,3.0,1.0,15
4,700,2000,0.8,0.05,3.0,1.0,20


Selecting the ID associated to the best fit

In [3]:
selected_IDs = [169]

## Computing and saving group duration distributions

In [8]:
#Input
IN_PATH = 'results/'

#Output
OUT_PATH = 'results-gduration/'
if not os.path.exists(OUT_PATH):
    os.makedirs(OUT_PATH)
    
for ID in selected_IDs:
    print(ID)
    #Reading the realisation with the given parameters
    Hs = read_edgelists_from_df(ID, pars_df, IN_PATH)
    print("read...")
    #Computing group durations
    durations = get_group_durations(Hs)
    #Saving
    OUT_FNAME = "gdurations_pars_id%i.csv"%ID
    pickle.dump( durations, open( OUT_PATH+OUT_FNAME, "wb" ) )

169
read...


## Computing and saving group aggregation and disaggregation matrices

First of all, I compute group times, that is for each group I save info on members and times of group creation and destruction

In [3]:
#Input
IN_PATH = 'results/'

#Output
OUT_PATH = 'results-gdisagg-mat/'
if not os.path.exists(OUT_PATH):
    os.makedirs(OUT_PATH)
    
for ID in selected_IDs:
    print(ID)
    #Reading the realisation with the given parameters
    Hs = read_edgelists_from_df(ID, pars_df, IN_PATH)
    print("Read. Computing groups and time...")
    #Computing times of groups start and end
    groups_and_times = get_group_times(Hs)
    print("Groups and times computed. Saving...")
    #Saving
    OUT_FNAME = "group_times_id%i.p"%ID
    pickle.dump(groups_and_times, open(OUT_PATH+OUT_FNAME, "wb" ) )

169
Read. Computing groups and time...
Groups and times computed. Saving...


Computing matrices using size of biggest sub-group and size of all sub-groups

In [ ]:
#Input
IN_PATH = 'results-gdisagg-mat/'

#Output
OUT_PATH = 'results-gdisagg-mat/'
if not os.path.exists(OUT_PATH):
    os.makedirs(OUT_PATH)

for ID in selected_IDs:
    print(ID)
    #Reading the realisation with the given parameters
    Hs = read_edgelists_from_df(ID, pars_df, 'results/')
    #Reading times of groups start and end for the selected realisation
    FNAME = "group_times_id%i.p"%ID
    groups_and_times = pickle.load(open(IN_PATH+FNAME, "rb" ) )
    print("Groups read. Computing matrices...")
    
    #Computing dis- and aggregation matrices related to biggest sub-group
    D, A = get_dis_agg_matrices(Hs, groups_and_times, max_k = 21, normed=True)
    #Converting them to dataframes
    df_D = dis_agg_matrix_to_df(D)
    df_A = dis_agg_matrix_to_df(A)
    print("Done. Saving...")
    #Saving
    OUT_FNAME = "D_id%i.csv"%ID
    df_D.to_csv(OUT_PATH+OUT_FNAME, header=True, index=False)   
    OUT_FNAME = "A_id%i.csv"%ID
    df_A.to_csv(OUT_PATH+OUT_FNAME, header=True, index=False)      
    
    #Computing dis- and aggregation matrices related to biggest sub-group
    D, A = get_full_dis_agg_matrices(Hs, groups_and_times, max_k = 21, normed=True)
    #Converting them to dataframes
    df_D = dis_agg_matrix_to_df(D)
    df_A = dis_agg_matrix_to_df(A)
    print("Done. Saving...")
    #Saving
    OUT_FNAME = "Dfull_id%i.csv"%ID
    df_D.to_csv(OUT_PATH+OUT_FNAME, header=True, index=False)   
    OUT_FNAME = "Afull_id%i.csv"%ID
    df_A.to_csv(OUT_PATH+OUT_FNAME, header=True, index=False)            

## Computing and saving "social memory"
I want to measure the density of known nodes in the group each node is chosing (when changing) compared to a random one.

In [ ]:
IN_PATH = 'results-gdisagg-mat/'

#Output
OUT_PATH = 'results-social-memory/'
if not os.path.exists(OUT_PATH):
    os.makedirs(OUT_PATH)

for ID in selected_IDs:
    print(ID)
    #Reading the realisation with the given parameters
    Hs = read_edgelists_from_df(ID, pars_df, 'results/')
    print("Hypergraphs read.")
    #Reading times of groups start and end for the selected realisation
    FNAME = "group_times_id%i.p"%ID
    groups_and_times = pickle.load(open(IN_PATH+FNAME, "rb" ) )
    print("Groups and times read.")
    #Computing the cumulative networks of contacts
    #To do this I need to convert Hs from a list to a dict indexed by time
    Hs = {k:v for k, v in enumerate(Hs)}
    Gs = get_cumulative_Gs_from_Hs(Hs)
    print("Cumulative contact graphs computed.")
    #I will also need the dictionary of groups at each time
    groups_at_t_dict = get_groups_dict_from_Hs(Hs)
    #Measuring 'social memory' dataframe
    memory_df = measure_social_memory(Hs, groups_at_t_dict, Gs, groups_and_times)
    print("Social memory dataframe computed. Saving...")
    #Saving
    OUT_FNAME = "social_memory_id%i.csv.gz"%ID
    memory_df.to_csv(OUT_PATH+OUT_FNAME, header=True, index=False, compression="gzip")            

## Computing and saving inter-event times

In [7]:
IN_PATH = 'results-gdisagg-mat/'

#Output
OUT_PATH = 'results-gduration/'
if not os.path.exists(OUT_PATH):
    os.makedirs(OUT_PATH)

for ID in selected_IDs:
    print(ID)
    #Reading times of groups start and end for the selected realisation
    FNAME = "group_times_id%i.p"%ID
    groups_and_times = pickle.load(open(IN_PATH+FNAME, "rb" ) )
    print("Groups and times read.")
    interevent_times = get_interevent_times(groups_and_times)
    print("Interevent times computed. Saving...")
    #Saving
    OUT_FNAME = "interevent_times_id%i.p"%ID
    pickle.dump( interevent_times, open( OUT_PATH+OUT_FNAME, "wb" ) )

169
Groups and times read.
Interevent times computed. Saving...
